In [7]:
import numpy as np
from katana import analytics
from katana.timer import StatTimer
from katana.local import DynamicBitset
from katana.distributed import (
    DistPropertyGraph,
    GluonSubstrate,
    MinReducer,
    WorkDetection,
    do_all,
    do_all_operator,
)
from katana import remote

In [8]:
# Connect to the Katana Server
c = remote.Client()

In [9]:
test_graph_version = 1
test_graph = "test_graph"

In [ ]:
graph: remote.Graph = c.get_graph(name=test_graph, version=test_graph_version)

In [ ]:
def dist_louvain(graph: DistPropertyGraph, vertex_opt, mod_threshold_per_round, mod_threshold_total, max_iter, min_graphsize):
    # Use defaults: Partition-aware, automatic metadata
    gluon = GluonSubstrate(graph)
    local_clusters = np.empty((len(graph),), dtype=np.uint32)
    updated_node_bitset = DynamicBitset()

    while True:
        # Run local louvain clustering (this includes coarsening phase, and cluster initialization)
        analytics.local_louvain_clustering(graph, local_custers, updated_node_bitset, vertex_opt,
                                        mod_threshold_per_round, mod_threshold_total, max_iter, min_graphsize)

        # Gluon communication of non-vertex programming model (pseudo)
        gluon.non_vertex_sync(MinReducer(local_clusters), updated_node_bitset,
                            GluonSubstrate.Destination, GluonSubstrate.Source, sync=True)

        # Check if louvain clustering completes
        if analytics.check_termination(graph, local_clusters, max_iter, min_graphsize):
            break

    return local_clusters

In [ ]:
# Run dit Louvain clustering

timer = StatTimer("Louvain cluestering")
timer.start()
clusters_output = graph.run(lambda g: dist_louvain(g, False, 0.0001, 0.0001, 10000, 100))
time.stop()

# Print cluster ids from nodes 0 to 10
if graph.user_to_local_id(0):
    for i in range(len(10)):
      print(f"{i} = {clusters_output[i]}")